# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re
from tqdm import tqdm
import random
from collections import Counter
from matplotlib import pyplot as plt

C:\Users\darmorezov\AppData\Local\Programs\Python\Python38-32\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


## Решение

In [ ]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('UTF8')))

In [ ]:
df.sample(n=5)

In [ ]:
def Conclusion_worker(analiz_res,norma):
    conclusion = 'Не определено'
    norma = re.sub(',','.',norma)
    if re.search('в\s*пределах\s*.+', norma):
        try:
            left_limit, right_limit = re.findall('\d+\.?\d*', norma)[0:2]
            conclusion = 'Ниже нормы' if float(analiz_res) < float(left_limit) else 'Выше нормы' if float(analiz_res) > float(right_limit) else 'В пределах нормы'
            return conclusion
        except Exception as error:
            print(f'В ходе работы произошла ошибка {error}')
            return 'Результат не определен'
    else:
        try:
            limit = re.findall('\d+\.?\d*',norma)[0]
            conclusion = 'Выше нормы' if float(analiz_res) > float(limit) else 'В пределах нормы'
            return conclusion
        except Exception as error:
            print(f'В ходе работы произошла ошибка {error}')
            return 'Результат не определен'

In [ ]:
df['Вывод'] = df.apply(lambda x: Conclusion_worker(x['Результат анализа'], x['Норматив']), axis=1)

In [ ]:
df.set_index('Показатель')

In [ ]:
df

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


## Решение

In [ ]:
fruits_box = (0,0,0,0,0,1,1,1,1)
oranges_number_list = []
for i in tqdm(range(0,1000000)):
    t = sum(random.sample(fruits_box, k=3))
    oranges_number_list.append(t)
cnt = dict(Counter(oranges_number_list))

In [ ]:
cnt

In [ ]:
explode = (0.1, 0, 0.15, 0)
fig, ax = plt.subplots()
ax.pie(cnt.values(), labels=cnt.keys(), autopct='%1.1f%%', shadow=True, explode=explode, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, rotatelabels=True)
ax.axis("equal")

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

## Решение

In [ ]:
details = (0,0,0,0,0,0,0,1,1,1)
count = 0
retries_count = 1000000
for i in tqdm(range(0,retries_count)):
    if random.sample(details, k=2) == [0,1]:
        count += 1
print(count/retries_count)